
# Nivel 3, Semana 7: El Solucionador Variacional de Eigenvalores Cuánticos (VQE)

**Audiencia:** Licenciatura / Avanzado

**Objetivo:** Generalizar el concepto del ciclo híbrido de la Semana 6 para resolver problemas físicos. Entender cómo mapear fermiones (electrones) a qubits usando el mapeo Jordan-Wigner y utilizar el algoritmo VQE para calcular la energía del estado fundamental de moléculas.

## 1. Puente desde la Semana 6: De QAOA a VQE

En la Semana 6, exploramos **QAOA** (Algoritmo Cuántico de Optimización Aproximada) para resolver el problema **Max-Cut**.

* **Meta:** Encontrar una cadena de bits $x$ que minimice una función de costo $C(x)$.
* **Método:** Construimos un Hamiltoniano de costo $H_C$ tal que su estado fundamental (energía más baja) codificaba la solución. Usamos un ansatz específico (capas alternas de unitaries de Costo y Mezclador) para encontrar ese estado fundamental.

**Semana 7: La Generalización**

QAOA es en realidad solo un caso específico de una clase más amplia de algoritmos llamados **VQE (Solucionador Variacional de Eigenvalores Cuánticos)**.

* **En QAOA:** *Inventamos* el Hamiltoniano $H_C$ para representar un problema de grafos creado por humanos.
* **En VQE:** La naturaleza nos da el Hamiltoniano $H$. Representa la energía de un sistema físico, como una molécula.
* **Meta:** Encontrar la **energía del estado fundamental** de esa molécula. Esta es la configuración más estable del sistema y determina tasas de reacción, energías de enlace y geometría molecular.

El ciclo central permanece igual: **Computadora cuántica** prepara un estado $\rightarrow$ **Estimator** mide la energía $\rightarrow$ **Computadora clásica** actualiza los parámetros.

## 2. El Problema Físico: Estructura Electrónica

Para simular una molécula (como Hidruro de Litio, LiH) en una computadora cuántica, tratamos de resolver la **Ecuación de Schrödinger**:

$$
H |\psi\rangle = E |\psi\rangle
$$

Específicamente, queremos encontrar el menor valor propio $E_0$ (la energía del estado fundamental).

### El reto de los fermiones

Los electrones son **fermiones**. Siguen el **Principio de Exclusión de Pauli**: ningún par de electrones puede ocupar el mismo estado cuántico. Matemáticamente, esto significa que su función de onda debe ser **antisimétrica**. Si intercambias dos electrones, la función de onda adquiere un signo negativo:

$$\Psi(r_1, r_2) = -\Psi(r_2, r_1)$$

Sin embargo, los qubits son distinguibles. $|0\rangle_1 |1\rangle_2$ no es lo mismo que $|1\rangle_1 |0\rangle_2$, y al intercambiarlos no se genera naturalmente un signo negativo. Para simular electrones usando qubits, necesitamos un "traductor" matemático.

## 3. Mapeo de Fermiones a Qubits: Jordan-Wigner

Necesitamos mapear la física de los electrones (orbitales vacíos u ocupados) a la física de los qubits (estados $|0\rangle$ o $|1\rangle$). El método estándar para esto es el **Mapeo Jordan-Wigner**.

### Paso 1: Definir los orbitales (Segunda Cuantización)
En vez de rastrear posiciones de electrones, rastreamos **orbitales**. Asignamos un índice $j$ a cada orbital disponible (orbital espacial $\times$ espín).

* **Ejemplo (4 espin-orbitales):** Imagina un modelo simplificado con 4 espin-orbitales, etiquetados $0, 1, 2, 3$.
    * Los orbitales 0 y 1 podrían ser la capa $1s$ (espín arriba/abajo).
    * Los orbitales 2 y 3 podrían ser la capa $2s$ (espín arriba/abajo).

Definimos operadores fermiónicos para cada orbital $j$:
* **Operador de creación ($a^\dagger_j$):** Crea un electrón en el orbital $j$.
* **Operador de aniquilación ($a_j$):** Elimina un electrón del orbital $j$.
* **Operador de número ($n_j = a^\dagger_j a_j$):** Cuenta si hay un electrón (1) o no (0).

### Paso 2: Mapear la ocupación a qubits
Esta parte es sencilla. Mapeamos la *presencia* de un electrón al estado de un qubit.
* **Qubit $j$ representa el orbital $j$.**
* **Estado $|0\rangle_j$:** El orbital $j$ está VACÍO.
* **Estado $|1\rangle_j$:** El orbital $j$ está OCUPADO.

**Ejemplo:**
* Estado con electrones en los orbitales 0 y 2: $|1\rangle_2 |0\rangle_1 |1\rangle_0$ (o $|101\rangle$).

### Paso 3: Mapear los operadores (La cadena de paridad)
Esta es la parte difícil. Debemos mapear los operadores $a_j$ y $a^\dagger_j$ a matrices de Pauli ($X, Y, Z$) de manera que se preserve la antisimetría (el signo negativo al intercambiar).

La transformación Jordan-Wigner lo logra adjuntando una "cadena" de operadores $Z$ a cada operador de creación/aniquilación. Esta cadena calcula la **paridad** de todos los qubits *debajo* del índice $j$.

**La fórmula:**
$$a^\dagger_j \rightarrow \left( \bigotimes_{k=0}^{j-1} Z_k \right) \otimes \sigma^+_j$$
$$a_j \rightarrow \left( \bigotimes_{k=0}^{j-1} Z_k \right) \otimes \sigma^-_j$$

* **$\sigma^\pm_j$:** Son los operadores de subida/bajada de qubit: $\sigma^+ = \frac{1}{2}(X-iY)$ y $\sigma^- = \frac{1}{2}(X+iY)$. Cambian el qubit $j$ (crean/destruyen el electrón).
* **Cadena $Z_k$:** Actúa en todos los qubits del 0 al $j-1$. Como $Z|1\rangle = -|1\rangle$, esta cadena cuenta el número de electrones "debajo" del orbital $j$ y aplica un signo negativo si ese número es impar. ¡Esta cadena "no local" impone la antisimetría!

**Ejemplo (4 orbitales):**
Mapeemos el operador de creación para el orbital 2, $a^\dagger_2$.
* Qubit objetivo: 2 (aplicar $\sigma^+_2$).
* Cadena de paridad: aplicar $Z$ a todos los qubits debajo de 2 (qubits 0 y 1).
* **Resultado:** $a^\dagger_2 \rightarrow \sigma^+_2 \otimes Z_1 \otimes Z_0 = \frac{1}{2}(X_2 - iY_2) Z_1 Z_0$.

### Paso 4: Transformar el Hamiltoniano
El Hamiltoniano molecular es una suma de términos que involucran estos operadores. Sustituimos los mapeos de qubits en el Hamiltoniano para obtener una suma de cadenas de Pauli.

**Ejemplo (Término de salto):**
Considera un término que representa un electrón saltando del orbital 0 al 2: $H_{hop} = a^\dagger_2 a_0 + a^\dagger_0 a_2$.
1. Mapea $a^\dagger_2 \rightarrow \sigma^+_2 Z_1 Z_0$.
2. Mapea $a_0 \rightarrow \sigma^-_0$.
3. Multiplica: $a^\dagger_2 a_0 = (\sigma^+_2 Z_1 Z_0)(\sigma^-_0) = \sigma^+_2 Z_1 (Z_0 \sigma^-_0)$.
   Como $Z_0 \sigma^-_0 = \sigma^-_0$ (¡verifícalo!), el término se convierte en $\sigma^+_2 Z_1 \sigma^-_0$.
4. Convierte a X/Y:
   $$\sigma^+_2 Z_1 \sigma^-_0 = \frac{1}{4} (X_2 - iY_2) Z_1 (X_0 + iY_0)$$
   $$= \frac{1}{4} (X_2 Z_1 X_0 + i X_2 Z_1 Y_0 - i Y_2 Z_1 X_0 + Y_2 Z_1 Y_0)$$

El Hamiltoniano final de qubits es una suma de estas cadenas de Pauli (como $X_2 Z_1 X_0$).

## 4. El Ansatz VQE

A diferencia de QAOA, donde la estructura del circuito está dictada por el grafo del problema, en VQE debemos elegir un circuito (Ansatz) que creamos pueda representar el estado fundamental de la molécula.

### Ansatz heurístico (Eficiente para hardware)
En el contexto de la computación cuántica, especialmente para algoritmos variacionales como VQE, el Ansatz heurístico es una función de prueba (circuito cuántico) diseñada con base en principios generales de operaciones de compuertas y restricciones del hardware, en vez de conocimiento físico o químico específico del problema.

Este ansatz ignora la química. En cambio, pregunta: "¿Qué circuito expresivo es fácil de ejecutar en esta computadora cuántica específica?"

* La estructura son varias capas de rotaciones de un solo qubit ($R_y, R_z$) seguidas de compuertas de entrelazamiento ($CNOT$) en un patrón repetitivo.
* Es fácil de implementar en hardware de corto plazo (NISQ), ya que tiene poca profundidad y es menos susceptible al ruido. Sin embargo, no conoce química y podría tener que buscar en un espacio masivo para encontrar el estado correcto (y es propenso a "mesetas estériles" como discutimos brevemente en la Semana 6).
* **Ejemplo:** `EfficientSU2` en Qiskit.

### Ansatz inspirado en química (UCCSD)
Es una función de prueba diseñada específicamente para modelar las interacciones electrónicas dentro de una molécula, incorporando principios físicos como conservación del número de electrones y simetría de espín. El más común es el Unitary Coupled Cluster Singles and Doubles (UCCSD).

* Estructura: El circuito se deriva matemáticamente para explorar solo estados que tienen sentido físico (conservando el número de electrones, espín, etc.). Comienza desde el estado Hartree-Fock (una aproximación simple donde los electrones llenan los orbitales de menor energía). Luego, el ansatz agrega "excitaciones" (aplica operaciones unitarias para mover electrones de orbitales ocupados a vacíos).
* Puede alcanzar alta precisión y típicamente encuentra el estado fundamental con menos parámetros que los circuitos genéricos. Sin embargo, tiene un modelo de circuito muy profundo que requiere muchas compuertas $CNOT$, lo que dificulta ejecutarlo en hardware ruidoso sin mitigación de errores.

## 5. Ejecución: El Principio Variacional

¿Cómo sabemos si encontramos el estado fundamental? Nos basamos en el **Principio Variacional** de la mecánica cuántica.

Para *cualquier* estado cuántico normalizado $|\psi(\theta)\rangle$, el valor esperado del Hamiltoniano siempre es mayor o igual a la verdadera energía del estado fundamental $E_0$:
$$\langle \psi(\theta) | H | \psi(\theta) \rangle \ge E_0$$

**El algoritmo** usando la librería Qiskit se puede desglosar en varios pasos:

1.  **Inicializar:** Elegir un Ansatz $U(\theta)$.
2.  **Medir:** Usar la primitiva **`Estimator`** de Qiskit para calcular el valor esperado (energía) $E(\theta) = \langle \psi(\theta) | H_{molécula} | \psi(\theta) \rangle$.
    * *Nota:* Como $H_{molécula}$ es una suma de cadenas de Pauli, el Estimator mide la base apropiada para cada cadena (X, Y o Z) y suma los resultados ponderados por los coeficientes $c_i$.
3.  **Optimizar:** Usar un optimizador clásico (SPSA o COBYLA) para cambiar $\theta$ y hacer $E(\theta)$ más pequeño.
4.  **Repetir:** Hasta que la energía deje de disminuir. El valor final es nuestra aproximación de la energía del estado fundamental.

## 6. Aplicación: La molécula LiH

Una de las aplicaciones donde se usa el ansatz inspirado en química es el Hidruro de Litio (LiH), que es lo suficientemente simple para simular y lo suficientemente complejo para ser interesante.

**Orbitales:** LiH tiene 4 electrones. Podemos mapear esto a 4 orbitales espaciales $\rightarrow$ 8 espin-orbitales $\rightarrow$ 8 qubits.

A menudo podemos "congelar" electrones del núcleo que no participan en el enlace para reducir el número de qubits (por ejemplo, a 4 o 6 qubits). Luego, podemos usar Qiskit Nature para generar el Hamiltoniano y aplicar el mapeo Jordan-Wigner. Como resultado, ejecutar este VQE produce una curva de Energía vs. Distancia Interatómica. El mínimo de esta curva nos indica la longitud de enlace de la molécula.
``

